# 🎙️ Hybrid Low-Resource TTS Dataset Synthesis Pipeline

This notebook implements the whitepaper workflow: **Format Conversion -> Denoising -> Trimming -> Normalization -> WPM Filtering -> Voice Cloning**.

### Usage Instructions:
1. **Configure Variables:** Set your base paths in the first code cell.
2. **Run Sequentially:** Execute cells in order. Dependencies are re-installed per step to avoid conflicts.
3. **Check Outputs:** Each step will output to a subdirectory within your `WORK_DIR`.

In [ ]:
import os

# ==========================================
# ⚙️ GLOBAL CONFIGURATION (ENV VARS)
# ==========================================

# Base directory for processing
os.environ["WORK_DIR"] = "/run/media/miro/endeavouros/processing_workspace"

# Input Data Sources
os.environ["RAW_AUDIO_SOURCE"] = "/run/media/miro/endeavouros/TTS_datasets/raw_audio"
os.environ["METADATA_CSV"] = "/run/media/miro/endeavouros/TTS_datasets/metadata.csv"
os.environ["REF_VOICES_DIR"] = "/run/media/miro/endeavouros/ref_voices"

# Processing Parameters
os.environ["TARGET_LUFS"] = "-20.0"
os.environ["WPM_STD_THRESHOLD"] = "2.0"
os.environ["MIN_TRIM_DURATION"] = "1.0"
os.environ["DEVICE"] = "cuda" # or cpu, mps

# Define pipeline subdirectories (Chain of custody)
base = os.environ["WORK_DIR"]
os.environ["DIR_CONVERTED"] = os.path.join(base, "1_converted")
os.environ["DIR_DENOISED"] = os.path.join(base, "2_denoised")
os.environ["DIR_TRIMMED"] = os.path.join(base, "3_trimmed")
os.environ["DIR_NORMALIZED"] = os.path.join(base, "4_normalized")
os.environ["DIR_FILTERED"] = os.path.join(base, "5_wpm_filtered")
os.environ["DIR_FINAL_VC"] = os.path.join(base, "6_vc_output")

print(f"Work Directory: {os.environ['WORK_DIR']}")
print(f"Target Device: {os.environ['DEVICE']}")

--- 
### Step 1: Format Standardization
Converts mixed audio formats (mp3, ogg, flv, etc.) to standard WAV format.

In [ ]:
# Reinstall specific dependencies for this step
!pip install pydub -q

import os
import shutil
from pydub import AudioSegment
from pathlib import Path

INPUT_DIR = os.environ.get("RAW_AUDIO_SOURCE")
OUTPUT_DIR = os.environ.get("DIR_CONVERTED")

SUPPORTED_FORMATS = {'.mp3', '.mp4', '.wma', '.ogg', '.flv', '.aac', '.m4a', '.amr', '.wav'}

def convert_dataset(input_path, output_path):
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    print(f"Converting from {input_path} -> {output_path}")
    
    for root, _, files in os.walk(input_path):
        for filename in files:
            file_path = os.path.join(root, filename)
            base, ext = os.path.splitext(filename)
            
            if ext.lower() not in SUPPORTED_FORMATS:
                continue

            target_file = os.path.join(output_path, f"{base}.wav")
            
            # If already wav, just copy to standardize location, else convert
            if ext.lower() == '.wav':
                shutil.copy2(file_path, target_file)
            else:
                try:
                    audio = AudioSegment.from_file(file_path)
                    audio.export(target_file, format='wav')
                    print(f"Converted: {filename}")
                except Exception as e:
                    print(f"Failed to convert {filename}: {e}")

convert_dataset(INPUT_DIR, OUTPUT_DIR)

---
### Step 2: Audio Denoising
Uses `resemble-enhance` to remove background noise and enhance speech quality.
*Note: This step can be resource intensive.*

In [ ]:
# Reinstall specific dependencies for this step
!pip install resemble-enhance torchaudio --upgrade -q

import os
import torch
import torchaudio
from pathlib import Path
from tqdm import tqdm
from resemble_enhance.enhancer.inference import denoise

INPUT_DIR = os.environ.get("DIR_CONVERTED")
OUTPUT_DIR = os.environ.get("DIR_DENOISED")
DEVICE = os.environ.get("DEVICE", "cpu")

def process_denoising(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    
    files = [f for f in os.listdir(input_dir) if f.endswith(".wav")]
    
    for fname in tqdm(files, desc="Denoising Audio"):
        in_path = os.path.join(input_dir, fname)
        out_path = os.path.join(output_dir, fname)
        
        if os.path.exists(out_path):
            continue
            
        try:
            dwav, sr = torchaudio.load(in_path)
            # Mono mix if necessary
            if dwav.shape[0] > 1:
                dwav = dwav.mean(dim=0)
            else:
                dwav = dwav.squeeze(0)
            
            # Denoise
            denoised_wav, new_sr = denoise(dwav, sr, DEVICE)
            
            # Save
            torchaudio.save(out_path, denoised_wav.unsqueeze(0), new_sr)
        except Exception as e:
            print(f"Error processing {fname}: {e}")

process_denoising(INPUT_DIR, OUTPUT_DIR)

---
### Step 3: Silence Trimming
Trims leading/trailing silence using `librosa`. Removes files that are too short after trimming.

In [ ]:
# Reinstall specific dependencies for this step
!pip install librosa soundfile -q

import os
import librosa
import soundfile as sf
from glob import glob
from tqdm import tqdm

INPUT_DIR = os.environ.get("DIR_DENOISED")
OUTPUT_DIR = os.environ.get("DIR_TRIMMED")
MIN_DURATION = float(os.environ.get("MIN_TRIM_DURATION", 1.0))

def trim_silence(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    files = glob(os.path.join(input_dir, '*.wav'))
    
    for fpath in tqdm(files, desc="Trimming Silence"):
        fname = os.path.basename(fpath)
        out_path = os.path.join(output_dir, fname)
        
        try:
            # Load
            audio, sr = librosa.load(fpath, sr=None, mono=True)
            
            # Trim
            trimmed_audio, _ = librosa.effects.trim(audio, top_db=40)
            duration = len(trimmed_audio) / sr
            
            # Filter short files
            if duration >= MIN_DURATION:
                sf.write(out_path, trimmed_audio, sr)
        except Exception as e:
            print(f"Error processing {fname}: {e}")

trim_silence(INPUT_DIR, OUTPUT_DIR)

---
### Step 4: Volume Normalization
Normalizes audio to a target loudness (e.g., -20.0 LUFS/dBFS) to ensure consistency.

In [ ]:
# Reinstall specific dependencies for this step
!pip install pydub -q

import os
from pydub import AudioSegment
from glob import glob
from tqdm import tqdm

INPUT_DIR = os.environ.get("DIR_TRIMMED")
OUTPUT_DIR = os.environ.get("DIR_NORMALIZED")
TARGET_DBFS = float(os.environ.get("TARGET_LUFS", -20.0))

def normalize_volume(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    files = glob(os.path.join(input_dir, '*.wav'))
    
    for fpath in tqdm(files, desc="Normalizing Volume"):
        fname = os.path.basename(fpath)
        out_path = os.path.join(output_dir, fname)
        
        try:
            audio = AudioSegment.from_wav(fpath)
            current_dbfs = audio.dBFS
            gain = TARGET_DBFS - current_dbfs
            
            normalized = audio.apply_gain(gain)
            normalized.export(out_path, format="wav")
        except Exception as e:
            print(f"Error normalizing {fname}: {e}")

normalize_volume(INPUT_DIR, OUTPUT_DIR)

---
### Step 5: WPM (Words Per Minute) Filtering
Filters out outliers based on speaking rate using the provided transcript CSV.

In [ ]:
# Reinstall specific dependencies for this step
!pip install pandas soundfile -q

import os
import re
import shutil
import pandas as pd
import soundfile as sf

INPUT_DIR = os.environ.get("DIR_NORMALIZED")
CSV_PATH = os.environ.get("METADATA_CSV")
OUTPUT_DIR = os.environ.get("DIR_FILTERED")
STD_THRESHOLD = float(os.environ.get("WPM_STD_THRESHOLD", 2.0))

def calculate_wpm(row, audio_dir):
    file_path = os.path.join(audio_dir, str(row['file_name']))
    # Handle missing extensions in CSV if necessary
    if not file_path.endswith('.wav') and not os.path.exists(file_path):
         file_path += ".wav"
         
    cleaned_transcript = re.sub(r'[^\w\s]', '', str(row['transcript'])).strip().lower()
    word_count = len(cleaned_transcript.split())

    if word_count == 0 or not os.path.exists(file_path):
        return 0.0

    try:
        with sf.SoundFile(file_path) as f:
            duration_min = (len(f) / f.samplerate) / 60.0
        return word_count / duration_min if duration_min > 0 else 0.0
    except:
        return 0.0

def filter_wpm(input_dir, csv_path, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    
    # Load CSV (Assuming Pipe separated based on provided script)
    try:
        df = pd.read_csv(csv_path, sep='|', names=['file_name', 'transcript'], header=None)
    except:
        # Fallback for standard CSV
        df = pd.read_csv(csv_path)
        
    print("Calculating WPM...")
    df['WPM'] = df.apply(lambda row: calculate_wpm(row, input_dir), axis=1)
    valid_data = df[df['WPM'] > 0]
    
    # Stats
    mean_wpm = valid_data['WPM'].mean()
    std_wpm = valid_data['WPM'].std()
    lower = mean_wpm - (STD_THRESHOLD * std_wpm)
    upper = mean_wpm + (STD_THRESHOLD * std_wpm)
    
    print(f"Mean WPM: {mean_wpm:.2f} | Range: {lower:.2f} - {upper:.2f}")
    
    filtered = valid_data[(valid_data['WPM'] >= lower) & (valid_data['WPM'] <= upper)]
    
    print(f"Copying {len(filtered)} valid files...")
    new_meta = []
    for _, row in filtered.iterrows():
        fname = str(row['file_name'])
        if not fname.endswith('.wav'): fname += ".wav"
        
        src = os.path.join(input_dir, fname)
        dst = os.path.join(output_dir, fname)
        
        if os.path.exists(src):
            shutil.copy2(src, dst)
            new_meta.append(f"{fname}|{row['transcript']}")
            
    # Save new metadata
    with open(os.path.join(output_dir, "metadata_filtered.csv"), "w") as f:
        f.write("\n".join(new_meta))

filter_wpm(INPUT_DIR, CSV_PATH, OUTPUT_DIR)

---
### Step 6: Voice Conversion (Synthesis)
Generates the final synthetic dataset using `ChatterboxVC` and the reference voices.

In [ ]:
# Reinstall specific dependencies for this step
# Note: Replace 'chatterbox' with specific git url if it is not on PyPI
!pip install torch torchaudio tqdm chatterbox -q

import os
import torch
import torchaudio as ta
from chatterbox.vc import ChatterboxVC
from tqdm import tqdm

INPUT_DIR = os.environ.get("DIR_FILTERED")
OUTPUT_BASE = os.environ.get("DIR_FINAL_VC")
REFS_FOLDER = os.environ.get("REF_VOICES_DIR")
DEVICE = os.environ.get("DEVICE", "cpu")

def run_voice_conversion():
    print(f"Loading Model on {DEVICE}...")
    try:
        model = ChatterboxVC.from_pretrained(DEVICE)
    except Exception as e:
        print(f"Failed to load model: {e}")
        return

    ref_voices = [f for f in os.listdir(REFS_FOLDER) if f.endswith('.wav')]
    source_files = [f for f in os.listdir(INPUT_DIR) if f.endswith('.wav')]

    for ref_voice in tqdm(ref_voices, desc="Processing Ref Voices"):
        voice_id = os.path.splitext(ref_voice)[0]
        ref_path = os.path.join(REFS_FOLDER, ref_voice)
        
        voice_out_dir = os.path.join(OUTPUT_BASE, voice_id)
        os.makedirs(voice_out_dir, exist_ok=True)

        for fname in tqdm(source_files, desc=f"Converting to {voice_id}", leave=False):
            out_file = os.path.join(voice_out_dir, fname)
            if os.path.exists(out_file):
                continue

            try:
                wav = model.generate(
                    audio=os.path.join(INPUT_DIR, fname),
                    target_voice_path=ref_path,
                )
                ta.save(out_file, wav, model.sr)
            except Exception as e:
                # Silent fail to keep loop moving
                continue

run_voice_conversion()